In [ ]:
from elasticsearch import Elasticsearch, helpers
import json

In [ ]:
from indexing.enums.index_names import IndexName
from indexing.mappings.mappings import ESMappings

In [ ]:
# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "put-your-password-here"

# Create the client instance
es = Elasticsearch(
    "https://localhost:9200",
    ca_certs="config/certs/http_ca.crt",
    basic_auth=("elastic", ELASTIC_PASSWORD),
    verify_certs=False
)

In [ ]:
es.info()

# Create BM25 index

In [ ]:
# create index
index_name = IndexName.UWATERLOO_COURSES_INDEX.value
mapping = ESMappings.get_mapping_from_index_name(index_name)
es.indices.create(index=index_name, body=mapping)

In [ ]:
# load data
with open("../scraping/contents/waterloo/output.json", "r") as f:
    data = json.load(f)

In [ ]:
def data_gen(data: list):
    for item in data:
        yield {
            "_index": IndexName.UWATERLOO_COURSES_INDEX,
            "_source": item
        }

In [ ]:
helpers.bulk(es, data_gen(data))

In [ ]:
es.search(index=IndexName.UWATERLOO_COURSES_INDEX.value, body={"from": 0, "size": 1, "query": {"match": {"courseDescription": "Machine Intelligence"}}})

In [ ]:
es.indices.delete(index=IndexName.UWATERLOO_COURSES_INDEX.value)